# Project <u>NLPP 4 OFDFT</u>
### Instructors
 - Michele Pavanello (in-person)
 - Xuecheng Shao (remote, from Rutgers)
### Objective
 - Resolve the issue of non-local pseudopotentials in OF-DFT and consequently be awarded the Nobel prize in Physics
### Background
 - PPs are made of local, $v_{loc}(r)$, and non-local, $\hat v_{NL}$, parts.
 - Local PPs are defined by <i>one-dimensional functions on a one-dimensional grid</i>
 - Non-local PPs are defined by projectors that are zero beyond a <b>cutoff radius (rcut)</b> 
 - OF-DFT can only use <b>local PPs</b>
### Idea and overview of workflow
 - Obtain the reference electron density by running KS-DFT (we will use QEpy, a python version of QE) with a standard PP.
 - Borrow the local part of the standard PP.
 - Define a <i>tunable</i> and <i>short range</i> (i.e., defined only for $r<rcut$) local PP, $\delta_{pp}(r)$, to be added to the local part of the standard PP, $v_{new}(r) = v_{loc}(r)+\delta_{pp}(r)$.
 - Minimize the density difference between the reference density and the OF-DFT density calculated with $v_{new}(r)$ using standard optimization techniques.
 - Ta-da! You have a new PP for OF-DFT that should be <b>much better</b> than the original PP.
### Requirements
 - Working `Python` 3.10 or 3.11
 - Installation of a few packages: `dftpy`, `qepy`, `matplotlib`
 - Packages are available on Quantum Mobile or through `pip install`
 - run the `qepy_graphene.ipynb` notebook to generate a reference density for Carbon (used in this example).

## 1. Set up

#### Load dftpy-related stuff

In [ ]:
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
from dftpy.functional.pseudo.psp import PSP
from dftpy.constants import environ

#### Load numpy and some scipy methods

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.signal import quadratic

## 2. Read the target density and PP file

In [ ]:
ions, rho_target, _ = io.read_all('rho.snpy')
grid = rho_target.grid
PP_list = {'C': './C.upf'}
# PP_list = {'C': './new.psp8'}

In [ ]:
MaxPoints=1000 # number of points in the one-dimensional PP function
PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list, MaxPoints=MaxPoints)
rho_ini = rho_target.copy()

## 3. Load the needed density functionals
 - `KEDF` is the non-interacting kinetic energy functional, $T_s[n]$
 - `HARTREE` is the electrostatic electronic energy, $E_H[n]=\frac{1}{2}\int \frac{n(\mathbf{r})n(\mathbf{r}^\prime)}{|\mathbf{r}-\mathbf{r}^\prime|}d\mathbf{r}d\mathbf{r}^\prime$
 - `XC` is the exchange-correlation functional, $E_{xc}[n]$

In [ ]:
# KE = Functional(type='KEDF',name='GGA_LKT')
KE = Functional(type='KEDF',name='TFvW', y=0.2)
XC = Functional(type='XC',name='LDA')
HARTREE = Functional(type='HARTREE')

## 4. Define the `total energy` functional

In [ ]:
evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)

## 5. Find optimal electron density with DFTpy's density optimizer

In [ ]:
optimization_options = {'econv' : 1e-6*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options, optimization_method = 'TN')
rho = opt.optimize_rho(guess_rho=rho_ini)

## 6. How much does the OF-DFT density deviate from the KS-DFT reference?
 - We use the expression
$$
\Delta n = \frac{1}{2}\int |n_\text{target}(\mathbf{r}) - n(\mathbf{r})| d\mathbf{r}
$$

In [ ]:
np.abs(rho_target-rho).integral()/2

 - We hope to improve this number with the new, custom PP.

## 7. Make your own tunable short range PP, $\delta_{PP}(r)$
 - the `a[i]` list is a list of parameters to be optimized, that is:
 $$\min_a\{ \Delta n \}$$
 - One option using polynomials up to 3rd order

In [ ]:
def delta_pp(r, rcut, a):
    d = r - rcut
    v = a[0]*d + a[1]*d**2 + a[2]*d**3
    v[r>rcut] = 0.0
    return v

 - Another option to add a term that goes like 1/r

In [ ]:
def delta_pp(r, rcut, a):
    d = r - rcut
    v = a[0]*d + a[1]*d**2 + a[2]*d**3 + a[3]*(1/(r+0.1)-1/(rcut+0.1))
    v[r>rcut] = 0.0
    return v


 - What other function would you like to try?

## 8. Represent the new tunable short range PP on the simulation grid

In [ ]:
def lpp2vloc(r, v, ions, grid, zval=0.0):
    engine = PSP(None)
    engine.r = r
    engine.v = v
    engine._zval = zval
    pseudo = LocalPseudo(grid = grid, ions=ions, PP_list={'C':engine}, MaxPoints=MaxPoints)
    pseudo.local_PP()
    return pseudo._vreal

## 9. Run the optimization

In [ ]:
grid = rho_target.grid
rcut = 1.3
r = np.linspace(0, rcut, 100)
# a = [-0.77646192, 0.33666482, 10.04099905]
a = np.zeros(5)

ext = Functional(type='EXT')
evaluator.UpdateFunctional(newFuncDict={'EXT': ext})

opt = Optimization(EnergyEvaluator=evaluator)

rho_ini = rho_target.copy()
environ['LOGLEVEL'] = 4
def delta_rho(a):
    v = delta_pp(r, rcut, a)
    ext.v = lpp2vloc(r, v, ions, grid)
    rho = opt.optimize_rho(guess_rho=rho_ini)
    rho_ini[:]=rho
    diff = 0.5 * (np.abs(rho - rho_target)).integral()
    print('aa:', a, diff)
    return diff

res = minimize(delta_rho, a, method='Powell', tol=1.0e-3)
environ['LOGLEVEL'] = 2

## 10. Copy resulting PP in a standard PP file format (to be used later)
 - First set the PP writer (called `engine` here)

In [ ]:
a = res.x
key = 'C'
r = PSEUDO.readpp.pp[key].r
vl = PSEUDO.readpp.pp[key].v
v = delta_pp(r, rcut, a)
v += vl

engine = PSP(None)
engine.r = r
engine.v = v
engine.info['atomicnum'] = 6
engine._zval = 4.0

 - `engine` allows us to plot the PP

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r,v)

 - Finally, write the PP

In [ ]:
engine.write('new.psp8')